In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms, utils
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)


Using device: cuda


In [6]:
import sys
!{sys.executable} -m pip install tqdm

'c:\Users\saiom\OneDrive\Desktop\college\Plant' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
import torch

print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU found")


Torch version: 2.5.1+cu121
CUDA available: True
GPU Name: NVIDIA GeForce RTX 3070 Laptop GPU


In [3]:

DATA_DIR = r"C:\Users\saiom\OneDrive\Desktop\college\Plant Disease Detection\New Plant Diseases Dataset(Augmented)"

train_dir = os.path.join(DATA_DIR, "train")
val_dir   = os.path.join(DATA_DIR, "valid")  
test_dir  = os.path.join(DATA_DIR, "test")

batch_size = 32
lr         = 1e-3
num_epochs = 10


In [5]:
data_transforms = {
    "train": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225])
    ]),
    "val": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225])
    ]),
    "test": transforms.Compose([  # same as val
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225])
    ])
}


In [14]:
assert os.path.isdir(train_dir), f"{train_dir} not found"
assert os.path.isdir(val_dir),   f"{val_dir} not found"

train_ds = datasets.ImageFolder(train_dir, transform=data_transforms["train"])
val_ds   = datasets.ImageFolder(val_dir,   transform=data_transforms["val"])

train_loader = DataLoader(train_ds, batch_size, shuffle=True,
                          num_workers=4, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size, shuffle=False,
                          num_workers=4, pin_memory=True)

print(f"Classes ({len(train_ds.classes)}): {train_ds.classes}")
print(f"#train: {len(train_ds)}, #val: {len(val_ds)}")


Classes (38): ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Cherry_(including_sour)___healthy', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust_', 'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___Leaf_scorch', 'Strawberry___healthy', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Toma

In [16]:
class TestDataset(Dataset):
    def __init__(self, folder, transform=None):
        self.folder     = folder
        self.file_list  = [f for f in os.listdir(folder) 
                            if f.lower().endswith((".jpg",".jpeg",".png"))]
        self.transform  = transform

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        fname = self.file_list[idx]
        path  = os.path.join(self.folder, fname)
        img   = Image.open(path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, fname

# Setup test loader
assert os.path.isdir(test_dir), f"{test_dir} not found"
test_ds     = TestDataset(test_dir, transform=data_transforms["test"])
test_loader = DataLoader(test_ds, batch_size, shuffle=False,
                         num_workers=0, pin_memory=torch.cuda.is_available())

print(f"#test images: {len(test_ds)}; examples: {test_ds.file_list[:5]}")


#test images: 33; examples: ['AppleCedarRust1.JPG', 'AppleCedarRust2.JPG', 'AppleCedarRust3.JPG', 'AppleCedarRust4.JPG', 'AppleScab1.JPG']


In [17]:
# Load pretrained VGG16
model = models.vgg16(pretrained=True)

# Freeze feature extractor
for p in model.features.parameters():
    p.requires_grad = False

# Replace final layer
num_classes = len(train_ds.classes)
model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)

# Move to device
model = model.to(device)


c:\Users\saiom\OneDrive\Desktop\college\Plant Disease Detection\venv310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\saiom\OneDrive\Desktop\college\Plant Disease Detection\venv310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [18]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)


In [19]:
def train_epoch(model, loader):
    model.train()
    running_loss = 0.0
    running_corrects = 0
    for imgs, labels in tqdm(loader, desc="Train"):
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outs = model(imgs)
        loss = criterion(outs, labels)
        loss.backward()
        optimizer.step()

        running_loss     += loss.item() * imgs.size(0)
        running_corrects += (outs.argmax(1) == labels).sum().item()
    return running_loss/len(loader.dataset), running_corrects/len(loader.dataset)

def eval_epoch(model, loader):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    with torch.no_grad():
        for imgs, labels in tqdm(loader, desc="Val  "):
            imgs, labels = imgs.to(device), labels.to(device)
            outs = model(imgs)
            loss = criterion(outs, labels)

            running_loss     += loss.item() * imgs.size(0)
            running_corrects += (outs.argmax(1) == labels).sum().item()
    return running_loss/len(loader.dataset), running_corrects/len(loader.dataset)


In [20]:
best_val_acc = 0.0
for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    train_loss, train_acc = train_epoch(model, train_loader)
    val_loss,   val_acc   = eval_epoch( model,   val_loader)

    print(f"Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f}")
    print(f" Val  Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_vgg16.pth")



Epoch 1/10


Val  : 100%|██████████| 550/550 [01:15<00:00,  7.31it/s]


Train Loss: 1.1513, Acc: 0.7627
 Val  Loss: 0.3554, Acc: 0.9141

Epoch 2/10


Val  : 100%|██████████| 550/550 [01:13<00:00,  7.46it/s]


Train Loss: 0.8407, Acc: 0.8445
 Val  Loss: 0.3166, Acc: 0.9269

Epoch 3/10


...

Train Loss: 0.5748, Acc: 0.9051
 Val  Loss: 0.2283, Acc: 0.9432

Epoch 9/10


Val  : 100%|██████████| 550/550 [01:18<00:00,  7.05it/s]


Train Loss: 0.5640, Acc: 0.9068
 Val  Loss: 0.1919, Acc: 0.9558

Epoch 10/10


Val  : 100%|██████████| 550/550 [01:14<00:00,  7.39it/s]

Train Loss: 0.5225, Acc: 0.9132
 Val  Loss: 0.2748, Acc: 0.9511


In [21]:
# If your test folder actually has class subfolders:
if any(os.path.isdir(os.path.join(test_dir, c)) for c in train_ds.classes):
    test_ds_labeled = datasets.ImageFolder(test_dir, transform=data_transforms["test"])
    test_loader_labeled = DataLoader(test_ds_labeled, batch_size, shuffle=False,
                                     num_workers=0, pin_memory=torch.cuda.is_available())

    model.load_state_dict(torch.load("best_vgg16.pth"))
    test_loss, test_acc = eval_epoch(model, test_loader_labeled)
    print(f"\nTest Accuracy (labeled): {test_acc*100:.2f}%")
else:
    print("Test folder is flat; skipping labeled accuracy.")


Test folder is flat; skipping labeled accuracy.


In [22]:
# Load best weights
model.load_state_dict(torch.load("best_vgg16.pth", map_location=device))
model.eval()

print("\nPredictions on flat test folder:")
with torch.no_grad():
    for imgs, fnames in test_loader:
        imgs  = imgs.to(device)
        outs  = model(imgs)
        preds = outs.argmax(1).cpu().tolist()
        for f, p in zip(fnames, preds):
            print(f"{f}  →  {train_ds.classes[p]}")


C:\Users\saiom\AppData\Local\Temp\ipykernel_9052\2510907838.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_vgg16.pth", map_locati


Predictions on flat test folder:
AppleCedarRust1.JPG  →  Apple___Cedar_apple_rust
AppleCedarRust2.JPG  →  Apple___Cedar_apple_rust
AppleCedarRust3.JPG  →  Apple___Cedar_apple_rust
AppleCedarRust4.JPG  →  Apple___Cedar_apple_rust
AppleScab1.JPG  →  Apple___Apple_scab
AppleScab2.JPG  →  Apple___Apple_scab
AppleScab3.JPG  →  Tomato___Target_Spot
CornCommonRust1.JPG  →  Corn_(maize)___Common_rust_
CornCommonRust2.JPG  →  Corn_(maize)___Common_rust_
CornCommonRust3.JPG  →  Corn_(maize)___Common_rust_
PotatoEarlyBlight1.JPG  →  Potato___Early_blight
PotatoEarlyBlight2.JPG  →  Potato___Early_blight
PotatoEarlyBlight3.JPG  →  Tomato___Septoria_leaf_spot
PotatoEarlyBlight4.JPG  →  Potato___Early_blight
PotatoEarlyBlight5.JPG  →  Potato___Early_blight
PotatoHealthy1.JPG  →  Potato___healthy
PotatoHealthy2.JPG  →  Potato___healthy
TomatoEarlyBlight1.JPG  →  Tomato___Early_blight
TomatoEarlyBlight2.JPG  →  Tomato___Late_blight
TomatoEarlyBlight3.JPG  →  Tomato___Target_Spot
TomatoEarlyBlight4.JPG

In [23]:
import torch
from torch.utils.data import DataLoader

def calculate_accuracy(model, test_dataset, device):
    model.eval()
    correct = 0
    total = 0

    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    accuracy = 100 * correct / total
    return accuracy


In [27]:
import os
import torch
from torchvision import datasets
from torch.utils.data import DataLoader

# 1) Load best model weights
model.load_state_dict(torch.load("best_vgg16.pth", map_location=device))
model.eval()

# 2) Inspect test_dir
subdirs = [d for d in os.listdir(test_dir) 
           if os.path.isdir(os.path.join(test_dir, d))]
if subdirs:
    # --- Labeled Test Set ---
    print("Detected class‐subfolders in test_dir; computing accuracy…")
    test_labeled_ds = datasets.ImageFolder(test_dir, transform=data_transforms["test"])
    test_labeled_loader = DataLoader(
        test_labeled_ds, batch_size, shuffle=False,
        num_workers=0, pin_memory=torch.cuda.is_available()
    )

    correct = total = 0
    with torch.no_grad():
        for imgs, labels in test_labeled_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            preds = outputs.argmax(1)
            correct += (preds == labels).sum().item()
            total   += labels.size(0)

    print(f"\n✅ Test Accuracy: {100*correct/total:.2f}%  ({correct}/{total})")

else:
    # --- Flat Test Folder ---
    print("No subfolders in test_dir; printing filename → prediction:")
    # reuse your existing test_loader
    with torch.no_grad():
        for imgs, fnames in test_loader:
            imgs = imgs.to(device)
            outputs = model(imgs)
            preds = outputs.argmax(1).cpu().tolist()
            for f, p in zip(fnames, preds):
                print(f"{f}  →  {train_ds.classes[p]}")


C:\Users\saiom\AppData\Local\Temp\ipykernel_9052\4051244287.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_vgg16.pth", map_locati

No subfolders in test_dir; printing filename → prediction:
AppleCedarRust1.JPG  →  Apple___Cedar_apple_rust
AppleCedarRust2.JPG  →  Apple___Cedar_apple_rust
AppleCedarRust3.JPG  →  Apple___Cedar_apple_rust
AppleCedarRust4.JPG  →  Apple___Cedar_apple_rust
AppleScab1.JPG  →  Apple___Apple_scab
AppleScab2.JPG  →  Apple___Apple_scab
AppleScab3.JPG  →  Tomato___Target_Spot
CornCommonRust1.JPG  →  Corn_(maize)___Common_rust_
CornCommonRust2.JPG  →  Corn_(maize)___Common_rust_
CornCommonRust3.JPG  →  Corn_(maize)___Common_rust_
PotatoEarlyBlight1.JPG  →  Potato___Early_blight
PotatoEarlyBlight2.JPG  →  Potato___Early_blight
PotatoEarlyBlight3.JPG  →  Tomato___Septoria_leaf_spot
PotatoEarlyBlight4.JPG  →  Potato___Early_blight
PotatoEarlyBlight5.JPG  →  Potato___Early_blight
PotatoHealthy1.JPG  →  Potato___healthy
PotatoHealthy2.JPG  →  Potato___healthy
TomatoEarlyBlight1.JPG  →  Tomato___Early_blight
TomatoEarlyBlight2.JPG  →  Tomato___Late_blight
TomatoEarlyBlight3.JPG  →  Tomato___Target_Sp

In [29]:
import os, re, torch

# Ensure model is in eval mode
model.eval()

# Build lookup: simplified → original class name
lookup = {}
for cls in train_ds.classes:
    # Remove non-letters, lower-case
    key = re.sub('[^a-z]', '', cls.lower())
    lookup[key] = cls

correct = total = 0

print("Starting inference + accuracy on flat test folder…\n")
with torch.no_grad():
    for imgs, fnames in test_loader:
        imgs = imgs.to(device)
        outs = model(imgs)
        preds = outs.argmax(1).cpu().tolist()

        for fname, p in zip(fnames, preds):
            # 1. Simplify filename (drop extension and digits, non-letters)
            base = os.path.splitext(fname)[0]  
            simple = re.sub('[^a-z]', '', base.lower())

            # 2. Lookup ground-truth
            true_cls = lookup.get(simple, None)
            pred_cls = train_ds.classes[p]

            if true_cls is not None:
                total += 1
                if pred_cls == true_cls:
                    correct += 1
            else:
                # no GT found for this file
                print(f"{fname}  →  {pred_cls}   (no GT match)")

# Report accuracy
if total:
    acc = 100 * correct / total
    print(f"\n✅ Test Accuracy: {acc:.2f}%   ({correct}/{total})")
else:
    print("\n⚠️ No ground-truth matches found—check your filename → class mapping.")


Starting inference + accuracy on flat test folder…

AppleCedarRust1.JPG  →  Apple___Cedar_apple_rust   (no GT match)
AppleCedarRust2.JPG  →  Apple___Cedar_apple_rust   (no GT match)
AppleCedarRust3.JPG  →  Apple___Cedar_apple_rust   (no GT match)
AppleCedarRust4.JPG  →  Apple___Cedar_apple_rust   (no GT match)
AppleScab1.JPG  →  Apple___Apple_scab   (no GT match)
AppleScab2.JPG  →  Apple___Apple_scab   (no GT match)
AppleScab3.JPG  →  Tomato___Target_Spot   (no GT match)
CornCommonRust1.JPG  →  Corn_(maize)___Common_rust_   (no GT match)
CornCommonRust2.JPG  →  Corn_(maize)___Common_rust_   (no GT match)
CornCommonRust3.JPG  →  Corn_(maize)___Common_rust_   (no GT match)
TomatoYellowCurlVirus1.JPG  →  Tomato___Tomato_Yellow_Leaf_Curl_Virus   (no GT match)
TomatoYellowCurlVirus2.JPG  →  Tomato___Tomato_Yellow_Leaf_Curl_Virus   (no GT match)
TomatoYellowCurlVirus3.JPG  →  Tomato___Tomato_Yellow_Leaf_Curl_Virus   (no GT match)
TomatoYellowCurlVirus4.JPG  →  Tomato___Tomato_Yellow_Leaf_Cur

In [30]:
# Save only the model’s learned parameters (recommended)
torch.save(model.state_dict(), "best_vgg16_weights.pth")

# (Optional) Save the full model (architecture + weights)
torch.save(model, "best_vgg16_full.pth")


In [ ]:

import heapq

# ─── CONFIG ─────────────────────────────────────────────────────────────
MODEL_WEIGHTS_PATH = "best_vgg16_weights.pth"
IMAGE_PATH         = r"C:\Users\saiom\OneDrive\Desktop\college\Plant Disease Detection\New Plant Diseases Dataset(Augmented)\straberryleaf.jpg"

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ─── REBUILD & LOAD MODEL ────────────────────────────────────────────────
num_classes = len(train_ds.classes)
model = models.vgg16(pretrained=False)
model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)
model.load_state_dict(torch.load(MODEL_WEIGHTS_PATH, map_location=DEVICE))
model.to(DEVICE)
model.eval()

# ─── INFERENCE PIPELINE (MATCHES VAL) ────────────────────────────────────
infer_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# ─── PREDICT FUNCTION ───────────────────────────────────────────────────
def predict_topk(image_path, model, transform, classes, k=3):
    img = Image.open(image_path).convert("RGB")
    x   = transform(img).unsqueeze(0).to(DEVICE)      
    with torch.no_grad():
        logits = model(x)[0]                           
        probs  = torch.softmax(logits, dim=0).cpu().tolist()
    # get top-k
    topk = heapq.nlargest(k, range(len(probs)), key=lambda i: probs[i])
    return [(classes[i], probs[i]) for i in topk]

# ─── RUN & PRINT ─────────────────────────────────────────────────────────
top3 = predict_topk(IMAGE_PATH, model, infer_transform, train_ds.classes, k=3)
print(f"Image: {IMAGE_PATH}\nTop‑3 predictions:")
for cls, p in top3:
    print(f"  {cls:30s} {p*100:6.2f}%")


c:\Users\saiom\OneDrive\Desktop\college\Plant Disease Detection\venv310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\saiom\OneDrive\Desktop\college\Plant Disease Detection\venv310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
C:\Users\saiom\AppData\Local\Temp\ipykernel_9052\2510371426.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-mode

Image: C:\Users\saiom\OneDrive\Desktop\college\Plant Disease Detection\New Plant Diseases Dataset(Augmented)\straberryleaf.jpg
Top‑3 predictions:
  Strawberry___Leaf_scorch       100.00%
  Apple___Apple_scab               0.00%
  Apple___Black_rot                0.00%
